# Preliminaries for Using the Marlowe Starter Kit (Chuẩn bị)

***Trước khi chạy sổ ghi chép này, bạn có thể muốn sử dụng chức năng "xóa đầu ra" của Jupyter để xóa kết quả thực hiện trước đó của sổ ghi chép này. Điều đó sẽ làm rõ hơn những gì đã được thực thi trong phiên hiện tại.***

Nền tảng phát triển web3 demeter.run cung cấp tiện ích mở rộng Cardano Marlowe Runtime có cài đặt các công cụ Marlowe và cung cấp các dịch vụ phụ trợ Marlowe Runtime và node Cardano. Nếu bạn không sử dụng demeter.run , hãy xem trang docker để biết hướng dẫn triển khai Marlowe Runtime bằng docker.

Sổ ghi chép này cung cấp hướng dẫn về cách thiết lập khóa ký và địa chỉ cho bộ khởi động này. Nó bao gồm các thông tin sau:

- Công cụ Marlowe
- Tạo địa chỉ và khóa ký
    - Faucet
    - Người cho vay
    - Người vay
- Nhận bài kiểm tra ADA
- Tài trợ cho địa chỉ của các bên
    - Sử dụng Daedalus hoặc ví trình duyệt web
    - Sử dụng faucet cục bộ tại dòng lệnh
    
[Một video hoạt động thông qua sổ ghi chép Jupyter này.](https://youtu.be/hGBmj9ZrYHs)

Bạn có thể đặt câu hỏi về Marlowe trong kênh #ask-marlowe trên IOG Discord hoặc đăng sự cố với bài học này lên danh sách sự cố cho kho lưu trữ github của Bộ công cụ khởi động Marlowe .

## Công cụ Marlowe

Ba quy trình công việc thay thế có sẵn để chạy các hợp đồng Marlowe:
1. Marlowe CLI (`marlowe-cli`) cho các thử nghiệm nhẹ với giao dịch Marlowe.
2. Marlowe Runtime CLI (`marlowe-runtime-cli`) cho các ứng dụng không phải web sử dụng dịch vụ phụ trợ Marlowe Runtime.
3. Marlowe Runtime Web (`marlowe-web-server`) cho các ứng dụng web sử dụng dịch vụ phụ trợ Marlowe Runtime.

Marlowe Runtime cung cấp nhiều dịch vụ xây dựng giao dịch, quản lý UTxO, truy vấn và gửi để sử dụng hợp đồng Marlowe: điều này giúp dễ dàng chạy hợp đồng Marlowe mà không cần tham khảo chi tiết về sổ cái Cardano và hợp đồng thông minh Plutus. Ngược lại, Marlowe CLI không hỗ trợ truy vấn và quản lý UTxO, vì vậy nó phù hợp nhất cho các nhà phát triển Cardano có kinh nghiệm.

![Tools for Running and Querying Marlowe Contracts](images/marlowe-tools.png)

### Truy cập vào node Cardano và Marlowe Runtime

Nếu chúng ta đang sử dụng tiện ích mở rộng Cardano Marlowe Runtime của demeter.run, thì chúng ta đã có quyền truy cập vào Cardano Node và Marlowe Runtime.Các lệnh sau sẽ đặt các biến môi trường cần thiết để sử dụng triển khai docker cục bộ trên các cổng mặc định. Nó cũng sẽ thiết lập một số biến môi trường bổ sung.

In [1]:
if [[ -z "$MARLOWE_RT_PORT" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"

CARDANO_NODE_SOCKET_PATH = ~/.local/share/containers/storage/volumes/marlowe-starter-kit_shared/_data/node.socket
CARDANO_TESTNET_MAGIC = 1


Lưu ý số magic mạng thử nghiệm:
- `preprod` = 1
- `preview` = 2

## Tạo địa chỉ và khóa

Cổng  [Cardano Developers Portal](https://developers.cardano.org/docs/stake-pool-course/handbook/keys-addresses/) chứa hướng dẫn tạo địa chỉ và khóa ký.

Bộ khởi động này sử dụng các địa chỉ sau:
- Một  ***tuỳ chọn*** cục bộ *Faucet* được sử dụng để tài trợ cho các bên tham gia hợp đồng Marlowe.
- Bên *cho vay* đối với các ví dụ trong bộ khởi động này.
- Bên *vay* cho các ví dụ trong bộ khởi động này.
- Bên *Mediator* cho vài ví dụ trong bộ khởi động này.

Hướng dẫn bên dưới trình bày chi tiết cách tạo khóa ký và địa chỉ cho các bên này. Giả định rằng một người có khóa ký và địa chỉ cho faucet và faucet đó đã được tài trợ bằng ADA thử nghiệm.

***QUAN TRỌNG:*** Thư  [keys/](keys/) mục chứa các khóa ký sẽ được tạo để tương tác với hợp đồng Marlowe. Nếu bạn xóa hoặc làm mất các tệp này, thì bạn cũng sẽ mất vĩnh viễn ADA thử nghiệm được lưu trữ tại các địa chỉ đó. Sao lưu các tệp này hoặc sau khi chạy hướng dẫn, hãy gửi phần còn lại của quảng cáo thử nghiệm trở lại ví lâu dài hơn hoặc trả lại cho faucet.

### The Faucet

***Bước này là tùy chọn nếu một người đang sử dụng ví đã được tài trợ bằng ADA thử nghiệm.***

Đặt tên tệp cho khóa ký và khóa xác minh của bên này.

In [2]:
FAUCET_SKEY=keys/faucet.skey
FAUCET_VKEY=keys/faucet.vkey

Tạo khóa nếu chúng chưa được tạo.

In [3]:
if [[ ! -e "$FAUCET_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$FAUCET_SKEY" \
    --verification-key-file "$FAUCET_VKEY"
fi

Tính toán địa chỉ của faucet trên mạng thử nghiệm.

In [4]:
FAUCET_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$FAUCET_VKEY" )
echo "$FAUCET_ADDR" > keys/faucet.address
echo "FAUCET_ADDR = $FAUCET_ADDR"

FAUCET_ADDR = addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Người cho vay

Đặt tên tệp cho khóa ký và khóa xác minh của bên này.

In [5]:
LENDER_SKEY=keys/lender.skey
LENDER_VKEY=keys/lender.vkey

Tạo khóa nếu chúng chưa được tạo.

In [6]:
if [[ ! -e "$LENDER_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$LENDER_SKEY" \
    --verification-key-file "$LENDER_VKEY"
fi

Tính toán địa chỉ của bên trên testnet.

In [7]:
LENDER_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$LENDER_VKEY" )
echo "$LENDER_ADDR" > keys/lender.address
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### Người đi vay

Đặt tên tệp cho khóa ký và khóa xác minh của bên này.

In [8]:
BORROWER_SKEY=keys/borrower.skey
BORROWER_VKEY=keys/borrower.vkey

Tạo khóa nếu chúng chưa được tạo.

In [9]:
if [[ ! -e "$BORROWER_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$BORROWER_SKEY"  \
    --verification-key-file "$BORROWER_VKEY"
fi

Tính toán địa chỉ của bên trên testnet.

In [10]:
BORROWER_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$BORROWER_VKEY" )
echo "$BORROWER_ADDR" > keys/borrower.address
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


### Người hòa giải

Đặt tên tệp cho khóa ký và khóa xác minh của bên này.

In [11]:
MEDIATOR_SKEY=keys/mediator.skey
MEDIATOR_VKEY=keys/mediator.vkey

Tạo khóa nếu chúng chưa được tạo.

In [12]:
if [[ ! -e "$MEDIATOR_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$MEDIATOR_SKEY"  \
    --verification-key-file "$MEDIATOR_VKEY"
fi

Tính toán địa chỉ của bên trên testnet.

In [13]:
MEDIATOR_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$MEDIATOR_VKEY" )
echo "$MEDIATOR_ADDR" > keys/mediator.address
echo "MEDIATOR_ADDR = $MEDIATOR_ADDR"

MEDIATOR_ADDR = addr_test1vr6tytqs3x8qgewhw89m3xrz58t3tqu2hfsecw0u06lf3hg052wsv


## Nhận bài kiểm tra ADA

Để thực hiện các giao dịch trên mạng Cardano, người ta cần tiền tệ bản địa ADA để trả phí và sử dụng làm tiền. Có các faucet dành cho mạng thử nghiệm công khai tại https://docs.cardano.org/cardano-testnet/tools/faucet nơi người ta có thể lấy ADA thử nghiệm hàng ngày.

Theo tùy chọn, có thể thuận tiện để quản lý tiền tập trung bằng [Daedalus wallet](https://docs.cardano.org/cardano-testnet/daedalus-testnet) hoặc một trong [các ví trên trình duyệt web](https://builtoncardano.com/ecosystem/wallets): đảm bảo chọn đúng mạng thử nghiệm công khai nếu sử dụng một trong các ví này.

Nếu bạn sẽ sử dụng một faucet cục bộ, thì hãy gửi ADA thử nghiệm đến địa chỉ faucet đã tạo trong phần trước. Nếu không, hãy gửi ADA thử nghiệm đến ví Daedalus hoặc trình duyệt web.

In [14]:
echo "FAUCET_ADDR = $FAUCET_ADDR"

FAUCET_ADDR = addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


## Tài trợ cho địa chỉ của các bên

Chúng ta sẽ tài trợ cho mỗi địa chỉ với 1000 ADA thử nghiệm.

In [15]:
echo "LENDER_ADDR = $LENDER_ADDR"
echo "BORROWER_ADDR = $BORROWER_ADDR"
echo "MEDIATOR_ADDR = $MEDIATOR_ADDR"

LENDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
BORROWER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d
MEDIATOR_ADDR = addr_test1vr6tytqs3x8qgewhw89m3xrz58t3tqu2hfsecw0u06lf3hg052wsv


### Sử dụng Daedalus hoặc ví

Nếu bạn đã có ví chứa ADA thử nghiệm, thì bạn chỉ cần gửi tiền đến địa chỉ của các khóa mà chúng ta đã tạo trong phần trước. Ảnh chụp màn hình bên dưới cho thấy việc sử dụng Daedalus để tài trợ cho địa chỉ người cho vay.

![Sending funds with Daedalus](images/daedalus-example.png)

### Sử dụng faucet cục bộ tại cửa sổ dòng lệnh

Người ta có thể sử dụng `cardano-cli` hoặc `marlowe-cli` gửi tiền đến một địa chỉ. Ở đây chúng ta sử dụng.

***Nếu bạn vừa nạp tiền `FAUCET_ADDR` bằng ADA, bạn có thể phải đợi vài phút trước khi giao dịch đó được xác nhận. Nếu lệnh bên dưới không thành công, hãy thử lại cho đến khi thành công.***

In [16]:
# Note that `FAUCET_ADDR` must have already been funded with test ada.

# 1 ada = 1,000,000 lovelace
ADA=1000000

# Send 1000 ada
AMOUNT=$((1000 * ADA))

# Execute the transaction.
marlowe-cli util fund-address \
 --lovelace "$AMOUNT" \
 --out-file /dev/null \
 --source-wallet-credentials "$FAUCET_ADDR":"$FAUCET_SKEY" \
 --submit 600 \
 "$LENDER_ADDR" "$BORROWER_ADDR" "$MEDIATOR_ADDR"

TxId "8461a35e612b38d4cb592e4ba1b7f13c2ff2825942d66e7200acc575cd4c8f1c"


Xem rằng các địa chỉ đã thực sự được tài trợ:

In [17]:
echo
echo "Lender @ $LENDER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"
echo

echo
echo "Borrower @ $BORROWER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"
echo

echo
echo "Mediator @ $MEDIATOR_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MEDIATOR_ADDR"
echo


Lender @ addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8461a35e612b38d4cb592e4ba1b7f13c2ff2825942d66e7200acc575cd4c8f1c     1        1000000000 lovelace + TxOutDatumNone


Borrower @ addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8461a35e612b38d4cb592e4ba1b7f13c2ff2825942d66e7200acc575cd4c8f1c     2        1000000000 lovelace + TxOutDatumNone


Mediator @ addr_test1vr6tytqs3x8qgewhw89m3xrz58t3tqu2hfsecw0u06lf3hg052wsv
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8461a35e612b38d4cb592e4ba1